# PR_8.1 Regressiemodellen in Machine Learning

## Opdracht

Great Outdoors wil graag weten hoeveel zij gaat verkopen (afhankelijke variabele) in het volgende jaar.
- Bedenk met welke onafhankelijke variabelen dit naar verwachting het beste voorspeld kan worden en zet deze samen met de afhankelijke variabele in één DataFrame.
- Pas waar nodig Dummy Encoding toe.
- Snijd dit DataFrame horizontaal en verticaal op de juiste manier.
- Train het regressiemodel.
- Evalueer de performance van je getrainde regressiemodel.

![Splitsprocess Train- en Testdata sets](../Assets/Week%208/Splitsprocess.jpg) 

### Onafhankelijke variabelen

1. Order_Date in Order_Header
2. Order_Number in order_Header

### Afhankelijke variabele 
1. Quantity

## Implementatie

### Library Imports

In [3]:
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy
import sqlite3
import csv


### Data import en samenvoeging

In [4]:
sales_con = None
crm_con = None
staff_con = None
GO_SALES_PRODUCT_FORECASTData = None
GO_SALES_INVENTORY_LEVELSData = None

try : 

        sales_con = sqlite3.connect("../Assets/Week 2/go_sales.sqlite")
        print("Connected to go_sales sqlite.")

   
        crm_con = sqlite3.connect("../Assets/Week 2/go_crm.sqlite")
        print("Connected to go_crm sqlite.")

  
        staff_con = sqlite3.connect("../Assets/Week 2/go_staff.sqlite")
        print("Connected to go_staff sqlite.")
        
        GO_SALES_PRODUCT_FORECASTData = pd.read_csv("../Assets/Week 2/GO_SALES_PRODUCT_FORECASTData.csv")
        print("Connected to GO_SALES_PRODUCT_FORECASTData")
        
        GO_SALES_INVENTORY_LEVELSData = pd.read_csv("../Assets/Week 2/GO_SALES_INVENTORY_LEVELSData.csv", header=0, index_col=False)
        print("Connected to GO_SALES_INVENTORY_LEVELSData.csv")


except sqlite3.Error as error: 
        print("Failed to read data from sqlite table", error)

Connected to go_sales sqlite.
Connected to go_crm sqlite.
Connected to go_staff sqlite.
Connected to GO_SALES_PRODUCT_FORECASTData
Connected to GO_SALES_INVENTORY_LEVELSData.csv


In [5]:
global product
product = pd.read_sql_query("SELECT * FROM {}".format("product"),sales_con)
product = product[product.columns.drop(list(product.filter(regex="TRIAL")))]

global product_type
product_type = pd.read_sql_query("SELECT * FROM {}".format("product_type"),sales_con)
product_type = product_type[product_type.columns.drop(list(product_type.filter(regex="TRIAL")))]

global product_line
product_line = pd.read_sql_query("SELECT * FROM {}".format("product_line"),sales_con)
product_line = product_line[product_line.columns.drop(list(product_line.filter(regex="TRIAL")))]

global order_header
order_header = pd.read_sql_query("SELECT * FROM {}".format("order_header"),sales_con)
order_header = order_header[order_header.columns.drop(list(order_header.filter(regex="TRIAL")))]

global order_details
order_details = pd.read_sql_query("SELECT * FROM {}".format("order_details"),sales_con)
order_details = order_details[order_details.columns.drop(list(order_details.filter(regex="TRIAL")))]

In [32]:
dataset = pd.merge(order_header, order_details, left_on="ORDER_NUMBER", how="inner", right_on="ORDER_NUMBER")
dataset = dataset.convert_dtypes()
dataset["PRODUCT_NUMBER"] = dataset["PRODUCT_NUMBER"].astype(int)
dataset = dataset.loc[:,["ORDER_DATE","PRODUCT_NUMBER","QUANTITY"]]
dataset["YEAR"] = pd.DatetimeIndex(dataset["ORDER_DATE"]).year
dataset["MONTH"] = pd.DatetimeIndex(dataset["ORDER_DATE"]).month
dataset = dataset.drop(["ORDER_DATE"], axis=1)

dataset = dataset.groupby(["PRODUCT_NUMBER","YEAR","MONTH"]).sum()


dataset


QUANTITY
PRODUCT_NUMBER YEAR MONTH          
1              2020 1           594
                    2           344
                    3           898
                    4          1328
                    5           640
...                             ...
115            2022 8           304
                    9          1020
                    10         1146
                    11          380
                    12          798

[3870 rows x 1 columns]

### One-hot encoding van onafhankelijke niet-numerieke variabelen

In [7]:
# pd.get_dummies(DATAFRAME.loc[:, [KOLOMMEN]])

### Verticaal en horizontaal snijden

In [8]:
# X_dataset = drop afhankelijk
# Y_dataset = selecter afhankelijk